Sent: 1
Sent: 2
Sent: 3


In [4]:
from keras.models import load_model  # TensorFlow is required for Keras to work
import cv2  # Install opencv-python
import numpy as np
import serial  # Install pyserial

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
with open("labels.txt", "r") as file:
    class_names = [line.strip() for line in file.readlines()]

# Define pest control categories and corresponding serial commands
pest_control_categories = {
    "Ants": "Vacuum System",
    "Bees": "Non-Target",
    "Bettles": "Vacuum System",
    "Caterpillar": "Vacuum System",
    "Earthworms": "Non-Target",
    "Earwig": "Vacuum System",
    "Grasshopper": "Fogging System",
    "Moth": "Light Trap",
    "Slug": "Non-Target",
    "Snail": "Non-Target",
    "Wasp": "Non-Target",
    "Weevil": "Vacuum System"
}

control_commands = {
    "Vacuum System": '2',
    "Light Trap": '3',
    "Fogging System": '1',
    "Non-Target": '4'
}

# Detect available cameras
available_cameras = []
for i in range(10):
    test_camera = cv2.VideoCapture(i)
    if test_camera.isOpened():
        print(f"Camera {i} is available.")
        available_cameras.append(i)
    test_camera.release()

if not available_cameras:
    print("No cameras found.")
    exit()

# Prompt user to select a camera
print("Select a camera from the available list:")
for index, cam in enumerate(available_cameras):
    print(f"{index}: Camera {cam}")

try:
    selected_camera_index = int(input("Enter the index of the camera to use: "))
    camera_index = available_cameras[selected_camera_index]
except (ValueError, IndexError):
    print("Invalid selection.")
    exit()

camera = cv2.VideoCapture(camera_index)

# Initialize serial communication (adjust 'COM9' to your serial port)
try:
    ser = serial.Serial('COM9', 9600)  # Replace 'COM9' with your port as needed
except serial.SerialException as e:
    print(f"Error: {e}")
    camera.release()
    cv2.destroyAllWindows()
    exit()

# Capture a single frame from the camera
ret, image = camera.read()

if ret:
    # Resize and normalize the image
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
    image_array = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)
    image_array = (image_array / 127.5) - 1  # Normalize

    # Display the image in a window
    cv2.imshow("Captured Image", image)
    cv2.waitKey(2000)  # Display the image for 2 seconds

    # Predict using the model
    prediction = model.predict(image_array)
    index = np.argmax(prediction)
    class_name = class_names[index]  # Predicted class name
    confidence_score = prediction[0][index] * 100  # Convert to percentage

    # Print prediction and confidence score
    print(f"Class: {class_name[2:]}  Confidence Score: {confidence_score:.2f}%")

    # Determine pest control action
    detect_class = pest_control_categories[class_name[2:]]
    command = control_commands[detect_class]

    # Send command via serial if available
    if command:
        ser.write(command.encode())
        print(f"Sent command '{command}' for '{detect_class}'")
else:
    print("Failed to capture image.")

# Cleanup
camera.release()
cv2.destroyAllWindows()
ser.close()

Camera 0 is available.
Camera 1 is available.
Select a camera from the available list:
0: Camera 0
1: Camera 1


Enter the index of the camera to use:  1


Error: could not open port 'COM9': PermissionError(13, 'Access is denied.', None, 5)
Failed to capture image.
